In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
import string
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from sklearn.metrics import confusion_matrix, f1_score
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases
import nltk
from nltk.collections import *
from nltk import FreqDist, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim
from wordcloud import WordCloud
plt.rcParams['figure.dpi'] = 300
sns.set(rc={"figure.dpi":500, 'savefig.dpi':500})
%matplotlib inline

In [6]:
df1 = pd.read_excel('hotjar_survey_response_export_2e84dc8e706e41a9b1aed68962b30f28_2002851_618030 (1).xlsx', index_col=0)
df1.head()

,User,Date Submitted,Country,Source URL,Device,Browser,OS,Is there anything preventing you from completing your purchase?,Would you like a response to your question or comment?
Number,,,,,,,,,
1789,c45329eb,2021-03-11 13:05:54,United States,https://www.gopetplan.com/purchase?c=ALUM229,desktop,Safari 14.0.3,Mac OS X 10.15.6,Need more information. Will premiums increase ...,No thanks
1788,f42d97df,2021-03-11 06:18:45,Ireland,https://www.gopetplan.com/purchase?trk_msg=7GO...,desktop,Chrome 89.0.4389,Mac OS X 10.15.7,can't find promo code,NaN
1787,cf424f15,2021-03-11 05:22:22,United States,https://www.gopetplan.com/purchase?trk_msg=4SA...,desktop,Safari 12.1.2,Mac OS X 10.12.6,is there a waiting period ?,Yes (please enter your email address) - trenia...
1786,93712644,2021-03-11 04:42:09,United States,https://www.gopetplan.com/purchase?c=AARP,desktop,Chrome 88.0.4324,Windows 8.1,Cost,No thanks
1785,c9987256,2021-03-11 02:48:21,United States,https://www.gopetplan.com/purchase?param=users...,desktop,Firefox 85.0,Mac OS X 10.15,yes,NaN


In [9]:
df =  df1[['User', 'Is there anything preventing you from completing your purchase?']]
df.columns = ['user', 'reviews']
df

,user,reviews
Number,,
1789,c45329eb,Need more information. Will premiums increase ...
1788,f42d97df,can't find promo code
1787,cf424f15,is there a waiting period ?
1786,93712644,Cost
1785,c9987256,yes
...,...,...
506,7d8a55a1,is there a discount for multiple pets?
505,77698e96,"comparing it to what i currently have now, if ..."
504,7509f26c,Question about policy limits


In [10]:
df.reviews = df.reviews.str.replace('[\n\t]', "", regex = True)
# df.Comment = df.Comment.astype('str')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1287 entries, 1789 to 502
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     1287 non-null   object
 1   reviews  1287 non-null   object
dtypes: object(2)
memory usage: 30.2+ KB


In [12]:
from transformers import pipeline
nlp=pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

In [13]:
nlp('customer service is absolutely horrible.')[0]

{'label': '1 star', 'score': 0.930723249912262}

In [14]:
to_sents = spacy.load("en_core_web_sm")
spacy_text_blob = SpacyTextBlob()
to_sents.add_pipe(spacy_text_blob)
star_senti = {'1 star':'Negative', '2 stars': 'Negative', '3 stars': 'Neutral', '4 stars': 'Positive', '5 stars': 'Positive'}
class trans_bert_score(object):
            
    def score_by_sentence(self, text):
        self.text = text
        self.sent_bert_label = {}
        sentences = to_sents(self.text)
        sents = [str(s).strip().lower() for s in sentences.sents]
        for i in sents:
            doc = nlp(i)
            self.sent_bert_label[i] = star_senti[doc[0]['label']]
        return self.sent_bert_label
    def label_bert_by_reviews(self, text):
        if text:
            doc = nlp(text[:1028])
            return doc[0]['label'] 

In [15]:
def text_preprocessing(text: list):
    nlp = spacy.load("en_core_web_sm")
    review_noun =[]
    
    text = [re.sub('\\S*@\\S*\\s?', '', sent) for sent in text]
    text = [re.sub('\\s+', ' ', sent) for sent in text]
    text = [re.sub("\\'", "", sent) for sent in text] 
    
    # only keep the nouns
    for review in text:
        noun = nlp(review)
        review_noun.append(' '.join([i.text for i in noun.noun_chunks]))
    print(review_noun[:5])
    # sentence to words
    review_words = []
    for sent in review_noun:
        review_words.append(gensim.utils.simple_preprocess(str(sent), deacc=True))
    print(review_words[:5])
    #remove numbers and lemmatization    
    docs = [[token for token in doc if not token.isnumeric()] for doc in review_words]
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    return docs 

In [16]:
def text_preprocessing2(text: list):
    nlp = spacy.load("en_core_web_sm")
    review_noun =[]
    
    text = [re.sub('\\S*@\\S*\\s?', '', sent) for sent in text]
    text = [re.sub('\\s+', ' ', sent) for sent in text]
    text = [re.sub("\\'", "", sent) for sent in text] 
    
    for review in text:
        sent = nlp(review)
        review_noun.append([token.lemma_ for token in sent if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'ADV']])
    print(review_noun[:5])
#   sentence to words
    review_words = []
    for sent in review_noun:
        review_words.append(gensim.utils.simple_preprocess(str(sent), deacc=True))
    print(review_words[:5])
    #remove numbers and lemmatization    
    docs = [[token for token in doc if not token.isnumeric()] for doc in review_words]
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    return docs

In [17]:
cs = pd.read_csv('cs_after_sentiments.csv')

In [18]:
reviews = cs.reviews.values.tolist()
reviews = [re.sub('\\S*@\\S*\\s?', '', sent) for sent in reviews]
reviews = [re.sub('\\s+', ' ', sent) for sent in reviews]
reviews = [re.sub("\\'", "", sent) for sent in reviews]
def reviews_to_words(reviews):
    for sent in reviews:
        yield(gensim.utils.simple_preprocess(str(sent), deacc=True))
list(reviews_to_words(['i like it', "I don't like it"]))
reviews_words = list(reviews_to_words(reviews))

In [19]:
bigrams = Phrases(reviews_words, min_count=1, threshold=10)
trigrams = Phrases(bigrams[reviews_words], min_count=1, threshold=10)
# bigrams.add_vocab(docs)
bigrams_mode = gensim.models.phrases.Phraser(bigrams)

In [20]:
trigrams['great_customer', 'service']

['great_customer_service']

In [26]:
reviews = df.reviews.values.tolist()
docs = text_preprocessing(reviews)
docs[:5]

['more information Will premiums the dog ages', 'promo code', 'a waiting period', 'Cost', '']
[['more', 'information', 'will', 'premiums', 'the', 'dog', 'ages'], ['promo', 'code'], ['waiting', 'period'], ['cost'], []]


[['more', 'information', 'will', 'premium', 'the', 'dog', 'age'],
 ['promo', 'code'],
 ['waiting', 'period'],
 ['cost'],
 []]

In [28]:
for i,j in enumerate(docs):
    if 'wa' in j:
        print(i, j)

38 ['the', 'zip', 'code', 'it', 'wa']


In [30]:
reviews[0]

'Need more information. Will premiums increase as the dog ages?'

In [31]:
stopwords_list = stopwords.words('english')
# stopwords_list.extend(['thats', 'fres', 'years', 'best', 'day', 'days','coin', 'coin_base', 'bit','coinbase',
#                            'time','would', 'week', 'anything', 'everything', 'nothing', 'lot'])
# "app", "petplan","company","will","day","much","I","It","even","t","review","time",
#                  "yesterday","This","pet","work","new","old","one","now","website","back","make","s","year"
#                  ,"way","try","anything","years","never","use","better","doesn","dont","week","star",'Claim','Customer'
#                   ,'Service','dog','Pet','pet','claims','w
final_docs = [[word for word in doc if word not in stopwords_list] for doc in docs]
final_docs = list(bigrams[final_docs])
final_docs = list(trigrams[final_docs])
final_docs = [[word for word in doc if word not in ['petplan', 
                                                    'plan', 'pet_plan']] for doc in final_docs]

In [32]:
x = []
for idx in range(len(final_docs)):
    for token in final_docs[idx]:
        if '_' in token:
            x.append(token)
freq_list = FreqDist(x)
print('\n-----------------')
freq_list.most_common(50)


-----------------


[('pet_insurance', 16),
 ('monthly_payment', 13),
 ('pre_existing', 11),
 ('credit_card', 9),
 ('another_dog', 7),
 ('gift_card', 4),
 ('insurance_plan', 4),
 ('phone_number', 2),
 ('long_time', 1),
 ('family_member', 1),
 ('direct_deposit', 1),
 ('payment_info', 1),
 ('card_number', 1),
 ('new_dog', 1),
 ('first_time', 1),
 ('another_way', 1),
 ('debit_card', 1),
 ('best_pet', 1),
 ('live_person', 1),
 ('monthly_premium', 1)]

In [34]:
bag_words = []
for doc in final_docs:
    bag_words.extend(doc)
freq_list = FreqDist(bag_words)
print('\n-----------------')
freq_list.most_common(20)


-----------------


[('discount', 83),
 ('code', 80),
 ('vet', 63),
 ('coverage', 59),
 ('policy', 53),
 ('price', 50),
 ('payment', 49),
 ('quote', 45),
 ('wellness', 39),
 ('cost', 36),
 ('period', 35),
 ('waiting', 31),
 ('error', 28),
 ('aarp', 27),
 ('condition', 25),
 ('information', 23),
 ('page', 22),
 ('website', 22),
 ('care', 21),
 ('visit', 21)]

In [35]:
reviews = df.reviews.values.tolist()
docs = text_preprocessing2(reviews)
docs[:5]

[['need', 'more', 'information', 'Will', 'premium', 'increase', 'dog', 'age'], ['can', 'find', 'promo', 'code'], ['waiting', 'period'], ['cost'], []]
[['need', 'more', 'information', 'will', 'premium', 'increase', 'dog', 'age'], ['can', 'find', 'promo', 'code'], ['waiting', 'period'], ['cost'], []]


[['need', 'more', 'information', 'will', 'premium', 'increase', 'dog', 'age'],
 ['can', 'find', 'promo', 'code'],
 ['waiting', 'period'],
 ['cost'],
 []]

In [36]:
stopwords_list = stopwords.words('english')
# stopwords_list.extend(['thats', 'fres', 'years', 'best', 'day', 'days','coin', 'coin_base', 'bit','coinbase',
#                            'time','would', 'week', 'anything', 'everything', 'nothing', 'lot'])
# "app", "petplan","company","will","day","much","I","It","even","t","review","time",
#                  "yesterday","This","pet","work","new","old","one","now","website","back","make","s","year"
#                  ,"way","try","anything","years","never","use","better","doesn","dont","week","star",'Claim','Customer'
#                   ,'Service','dog','Pet','pet','claims','w
final_docs = [[word for word in doc if word not in stopwords_list] for doc in docs]
final_docs = list(bigrams[final_docs])
final_docs = list(trigrams[final_docs])
final_docs = [[word for word in doc if word not in ['petplan', 
                                                    'plan', 'pet_plan']] for doc in final_docs]

In [37]:
x = []
for idx in range(len(final_docs)):
    for token in final_docs[idx]:
        if '_' in token:
            x.append(token)
freq_list = FreqDist(x)
print('\n-----------------')
freq_list.most_common(50)


-----------------


[('would_like', 29),
 ('pet_insurance', 14),
 ('monthly_payment', 13),
 ('make_sure', 10),
 ('pre_existing', 9),
 ('credit_card', 8),
 ('insurance_plan', 4),
 ('gift_card', 3),
 ('submit_claim', 3),
 ('long_time', 2),
 ('go_back', 2),
 ('phone_number', 2),
 ('family_member', 1),
 ('last_month', 1),
 ('still_want', 1),
 ('money_back', 1),
 ('direct_deposit', 1),
 ('would_rather', 1),
 ('could_go', 1),
 ('last_year', 1),
 ('payment_info', 1),
 ('call_back', 1),
 ('card_number', 1),
 ('new_dog', 1),
 ('hit_submit', 1),
 ('first_time', 1),
 ('good_plan', 1),
 ('take_care', 1),
 ('debit_card', 1),
 ('live_person', 1),
 ('monthly_premium', 1),
 ('right_away', 1),
 ('much_time', 1),
 ('find_information', 1),
 ('would_help', 1)]

In [38]:
bag_words = []
for doc in final_docs:
    bag_words.extend(doc)
freq_list = FreqDist(bag_words)
print('\n-----------------')
freq_list.most_common(20)


-----------------


[('cover', 110),
 ('pay', 93),
 ('discount', 83),
 ('need', 81),
 ('want', 79),
 ('code', 68),
 ('dog', 63),
 ('insurance', 62),
 ('coverage', 61),
 ('vet', 61),
 ('pet', 58),
 ('policy', 54),
 ('price', 52),
 ('know', 52),
 ('see', 51),
 ('quote', 50),
 ('payment', 47),
 ('cost', 46),
 ('go', 46),
 ('get', 41)]

In [40]:
words_re = re.compile("|".join(['(?i)would like']))

container = []
for i in reviews:
    if words_re.search(i):
        container.append(i)
         

In [41]:
container

['I would like a different start date since I am covered by my other ins until 3/24. I would prefer a start date of @ 3/20. Also, what is cost per month without the AARP promo code?',
 'I would like to talk to the veterinarian that looks after my "boy kitties". Make sure they accept Petplan.',
 'i would like to know the places where i can bring my dog in case of a illness',
 'I would like more details about coverage',
 'I would like to save my quote.',
 'I would like to start the insurance on March 1. Not today. I have other insurance that is effective until March.',
 'Yes, I would like a wellness plan also',
 'i would like to see comprehensive contract, evidence of coverage.',
 'Comparing to others.  Would like to see sample policy.',
 'I would like to see the monthly payment before I put in my card number.',
 'yes, i would like to get the full pet plan brochure to see exactly what is covered, procedures, and also what does 80% reimbursement mean?',
 "I'm a member of AARP - would like

In [42]:
to_dict = gensim.corpora.Dictionary(final_docs)
corpus = [to_dict.doc2bow(doc) for doc in final_docs]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=to_dict,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           eta = 'auto',
                                           per_word_topics=True)

In [43]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score, higher the better
coherence_model_lda = CoherenceModel(model=lda_model, texts=final_docs, dictionary=to_dict, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.42120608387706

Coherence Score:  0.45546538400101044


In [44]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, to_dict)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.220137 -0.136397       1        1  33.612769
3     -0.035853  0.167419       2        1  30.992034
1      0.047952  0.109751       3        1  17.984323
0      0.208038 -0.140773       4        1  17.410874, topic_info=           Term       Freq      Total Category  logprob  loglift
37          pay  78.000000  78.000000  Default  30.0000  30.0000
117         pet  52.000000  52.000000  Default  29.0000  29.0000
43         want  73.000000  73.000000  Default  28.0000  28.0000
6          code  49.000000  49.000000  Default  27.0000  27.0000
11         cost  33.000000  33.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
415  deductible   8.332099  19.360998   Topic4  -4.6474   0.9049
4          need  12.663876  66.133993   Topic4  -4.2288   0.0951
94          yet   8.043945  28.211792   Topic4  -4.6826   0.4933
67     complete   6.726820  13.880580   Topic4  -4.8614   1.0237
214        sure   6.838702  17.296200   Topic4  -4.8449   0.8202

[215 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
192       2  0.377336   accept
192       3  0.566005   accept
352       3  0.741481  account
89        2  0.206242      add
89        3  0.790596      add
...     ...       ...      ...
229       1  0.491908     year
229       2  0.491908     year
94        2  0.708923      yet
94        4  0.283569      yet
109       3  0.984138      zip

[225 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 2, 1])